In [1]:
%load_ext autoreload
%autoreload 2

In [46]:
import sys
sys.path.append('../')
from BirdCard import BirdCard
from BonusCard import BonusCard
from Birdfeeder import Birdfeeder
from Player import Player
from imagepaths import habitat_paths, food_paths, nest_paths
from load_data_functions import load_birdcards, load_bonuses
from Gameplay_Constants import FOOD_TYPES, BIRDCARD_FOOD_MAPPING, BIRDFEEDER_FACES

from xml.dom.minidom import getDOMImplementation, parseString
import os
from textwrap import wrap

WORKDIR = r'C:\Users\osmou\Documents\GitHub\wingspan-bird-brain\src'

birdcards = load_birdcards()
bonuscards = load_bonuses()

# Helpers

In [9]:
def createImgTag(src, width=20, height=20, style=''):
    return parseString(f'<img src="{src}" width="{width}" height="{height}" style="{style}"></img>').documentElement

# Creators

In [42]:
#=======================================================================================================================
def createBirdCardDiv(birdcard):
    assert type(birdcard) is BirdCard

    topDiv = parseString(f'<div class="birdcard" style="border: 1px solid black; font-family: cursive; font-weight: bold; display: inline-block;"> <u>{birdcard.common_name}</u> </div>').documentElement
    topDiv.appendChild(parseString(f'<div class="bird info" style="font-weight: normal;">VP: {birdcard.victory_points}, Wingspan: {birdcard.wingspan}</div>').documentElement)

    # Habitats
    habitatDiv = parseString('<div class="habitats"></div>').documentElement
    if birdcard.forest:
        habitatDiv.appendChild(createImgTag(os.path.join(WORKDIR, habitat_paths['Forest'])))
    if birdcard.grassland:
        habitatDiv.appendChild(createImgTag(os.path.join(WORKDIR, habitat_paths['Grassland'])))
    if birdcard.wetland:
        habitatDiv.appendChild(createImgTag(os.path.join(WORKDIR, habitat_paths['Wetland'])))
    topDiv.appendChild(habitatDiv)

    # Food Cost
    sep = '/' if birdcard.slash_food_cost else '+'
    totalFoodCost = birdcard.total_food_cost
    foodAdded = 0
    foodCostDiv = parseString('<div class="foodCost" style="display: flex;"></div>').documentElement
    for food in FOOD_TYPES:
        cnt = birdcard.__dict__[BIRDCARD_FOOD_MAPPING[food]]
        for _ in range(cnt):
            foodCostDiv.appendChild(createImgTag(os.path.join(WORKDIR, food_paths[food])))
            foodAdded += 1
            if foodAdded < totalFoodCost:
                # foodCostDiv.appendChild(doc.createTextNode(sep))
                foodCostDiv.appendChild(parseString(f'<div>{sep}</div>').documentElement)
    topDiv.appendChild(foodCostDiv)

    # Nest type and eggs
    nestDiv = parseString('<div class="nest"></div>').documentElement
    nestDiv.appendChild(createImgTag(os.path.join(WORKDIR, nest_paths[birdcard.nest_type])))
    for _ in range(birdcard.laideggs):
        nestDiv.appendChild(createImgTag(os.path.join(WORKDIR, 'ui', 'images', 'game_egg.png'), height=25, width=25))
    for _ in range(birdcard.egg_capacity - birdcard.laideggs):
        nestDiv.appendChild(createImgTag(os.path.join(WORKDIR, 'ui', 'images', 'game_smallegg.png')))
    topDiv.appendChild(nestDiv)

    # Power text
    topDiv.appendChild(parseString(f'<div class="power text" style="font-weight: normal;">{birdcard.power_text}</div>').documentElement)

    return topDiv

#=======================================================================================================================
def createBonusCardDiv(bonuscard):
    assert type(bonuscard) is BonusCard
    
    topDiv = parseString(f'''
        <div class="bonuscard" style="border: 1px solid black; font-family: cursive; display: inline-block; text-align: center;"> 
            <b><u>{bonuscard.name}</u></b><br></br>
            {'<br></br>'.join(wrap(bonuscard.condition, 40))}<br></br>
            {'<br></br>'.join(wrap(bonuscard.explanatory_text, 40))}<br></br>
            {bonuscard.vp}<br></br>
            {bonuscard.percent}% of cards<br></br>
        </div>
    ''').documentElement

    return topDiv

#=======================================================================================================================
def createBirdfeederDiv(birdfeeder):
    assert type(birdfeeder) is Birdfeeder

    topDiv = parseString('<div class="birdfeeder" style="font-family: cursive; display: inline-block; border: 1px solid black"><b><u>Birdfeeder</u></b></div>').documentElement
    inFeederDiv = parseString('<div class="infeeder" style="border: 1px solid gray;"></div>').documentElement
    topDiv.appendChild(inFeederDiv)
    for dice in birdfeeder.food:
        inFeederDiv.appendChild(createImgTag(os.path.join(WORKDIR, food_paths[dice]), width=40, height=40))
    outOfFeederDiv = parseString('<div class="outoffeeder"></div>').documentElement
    topDiv.appendChild(outOfFeederDiv)
    for _ in range(5-len(birdfeeder.food)):
        outOfFeederDiv.appendChild(createImgTag(os.path.join(WORKDIR, 'ui/images/game_die.png'), height=40, width=40))

    return topDiv

#=======================================================================================================================
def createHabitatEnv(player, habitat):
    assert habitat in ['forest', 'grassland', 'wetland']
    assert type(player) is Player

    topDiv = parseString(f'<div class="{habitat}" style="font-family: cursive; display: flex; border: 1px solid black"></div>').documentElement
    topDiv.appendChild(parseString(f'<div style="display: inline-block"> <b>{habitat.upper()}</b> {createImgTag(os.path.join(WORKDIR, habitat_paths[habitat.capitalize()]), height=40, width=40).toxml()} </div>').documentElement)
    playerHab = player.gamemat.habitats[habitat]
    for card in playerHab:
        topDiv.appendChild(createBirdCardDiv(card))
        
    return topDiv

In [75]:
# Create blank HTML
impl = getDOMImplementation()
doc = impl.createDocument(None, 'gui', None)
html = doc.documentElement

# # Add test birdcard
# birdcard = birdcards[0]
# birdcard.laideggs = 1 # add a laid egg
# html.appendChild(createBirdCardDiv(birdcard))

# # Add test bonuscard
# bonuscard = bonuscards[0]
# html.appendChild(createBonusCardDiv(bonuscard))

# # Add test birdfeeder
# birdfeeder = Birdfeeder()
# birdfeeder.take(birdfeeder.food[0]) # remove some food
# birdfeeder.take(birdfeeder.food[-1])
# html.appendChild(createBirdfeederDiv(birdfeeder))

# # Add Forest
# player = Player(birdcards=birdcards[:2], bonuscards=bonuscards[-1])
# player.gamemat.habitats['forest'].append(birdcards[3])
# player.gamemat.habitats['forest'].append(birdcards[4])
# player.gamemat.habitats['forest'].append(birdcards[5])
# # player.gamemat.habitats['forest'].append(birdcards[6])
# # player.gamemat.habitats['forest'].append(birdcards[7])
# html.appendChild(createHabitatEnv(player, 'forest'))

# # Add grassland
# player.gamemat.habitats['grassland'].append(birdcards[8])
# player.gamemat.habitats['grassland'].append(birdcards[9])
# player.gamemat.habitats['grassland'].append(birdcards[10])
# player.gamemat.habitats['grassland'].append(birdcards[11])
# player.gamemat.habitats['grassland'].append(birdcards[12])
# html.appendChild(createHabitatEnv(player, 'grassland'))

# # Add wetland
# # player.gamemat.habitats['wetland'].append(birdcards[13])
# # player.gamemat.habitats['wetland'].append(birdcards[14])
# # player.gamemat.habitats['wetland'].append(birdcards[15])
# # player.gamemat.habitats['wetland'].append(birdcards[16])
# # player.gamemat.habitats['wetland'].append(birdcards[17])
# html.appendChild(createHabitatEnv(player, 'wetland'))

# Write to HTML
with open('test.html', 'w') as f:
    f.write(doc.toxml())